Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

# Let's Wrangle and create our subset

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
# location of CSV: (DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

In [3]:
def make_features(df):
  return df

def wrangle (filepath):
  df = pd.read_csv(filepath,
                   parse_dates=['SALE DATE'],
                   index_col='SALE DATE')

# Replace column name spaces to underscores
  df.columns = [col.replace(' ', '_') for col in df]

# Reformat Sale Price column to integer from string
  df['SALE_PRICE'] = (
      df['SALE_PRICE']
      .str.replace('$', '')
      .str.replace('-', '')
      .str.replace(',', '')
      .astype(int)
  )
  
# Reformat BOROUGH from numeric to string to more accurately
# reflect that it is categorical
  df['BOROUGH'] = df['BOROUGH'].astype(str)

# Reduce cardinality of the NEIGHBORHOOD feature

# Produce list of top 10 neighborhoods
  top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# Replace results outside of the top 10 neighborhoods with OTHER
  df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

# Remove high-cardinality categorical variables
  high_cardinality = [col for col in df.select_dtypes('object').columns
                      if df[col].nunique() > 50]
  df.drop(columns=high_cardinality, inplace=True)

# Remove SALE_PRICE outliers

  df = df[(df['SALE_PRICE'] > 100000) &
          (df['SALE_PRICE'] < 2000000)]


  df = make_features(df)

  return df

In [4]:
# Use our function to wrangle the data into a workable format
df = wrangle(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

In [5]:
# Create our desired subset from the newly wrangled dataset
subset = (df.loc[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'])

# Drop the column full of NaN values
subset.drop(columns='EASE-MENT', inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
# Check our work thus far
subset.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,620000


# Let's Split our data for Train/Test

In [7]:
# Separate our target vector from feature matrix
target = 'SALE_PRICE'
y = subset[target]
X = subset.drop(columns=target)

In [8]:
# Separate training data from test data (using date of sale)
cutoff = '2019-04-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

# Let's establish our baseline

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = [y_train.mean()] * len(y_train)
print('Mean sale price for a single family home:', y_train.mean())
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Mean sale price for a single family home: 621573.7423214999
Baseline MAE: 214721.52773001452


# Let's build our models

In [10]:
from sklearn.linear_model import LinearRegression, Ridge
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [11]:
# Instantiate
ohe = OneHotEncoder(use_cat_names=True)
skb = SelectKBest(k=10)

In [12]:
# Fit the transformer to our training data
ohe.fit(X_train)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


OneHotEncoder(cols=['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
                    'TAX_CLASS_AT_PRESENT'],
              drop_invariant=False, handle_missing='value',
              handle_unknown='value', return_df=True, use_cat_names=True,
              verbose=0)

In [13]:
# Transform the data, but do not refit on test
XT_train = ohe.transform(X_train)
XT_test = ohe.transform(X_test)

In [14]:
# Further transformation
skb.fit(XT_train, y_train)
XTT_train = skb.transform(XT_train)
XTT_test = skb.transform(XT_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 4 11 22] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [15]:
model_lr = LinearRegression()
model_lr.fit(XTT_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
model_r = Ridge(alpha=1, normalize=True)
model_r.fit(XT_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

# Let's see our readouts

In [17]:
# For Linear Regression
print('Linear Regression training MAE:', mean_absolute_error(y_train, model_lr.predict(XTT_train)))
print('Linear Regression test MAE:', mean_absolute_error(y_test, model_lr.predict(XTT_test)))

Linear Regression training MAE: 181279.12478514545
Linear Regression test MAE: 174090.2687519857


In [18]:
# For Ridge Regression
print('Ridge training MAE:', mean_absolute_error(y_train, model_r.predict(XT_train)))
print('Ridge test MAE:', mean_absolute_error(y_test, model_r.predict(XT_test)))

Ridge training MAE: 174532.53995100924
Ridge test MAE: 174863.64578581622
